In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')

    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews

    !pip install joypy

    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime
import gc
import pandas as pd
import joypy

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.datasets as datasets

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import timedelta

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/dist

In [28]:
root_audios_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv"
root_recorders_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_recorders.csv"
root_ai_Zamuro = f"{root}/Zamuro/AI_Caract/AI_Zamuro.csv"

audios = pd.read_csv(root_audios_Zamuro, index_col=0)
recorders = pd.read_csv(root_recorders_Zamuro, index_col=0)
df_ai = pd.read_csv(root_ai_Zamuro, index_col=0)
df_ai.dropna(inplace=True)
df_ai.set_index("file",inplace=True, drop=True)
df_ai.drop(columns=["Date"], inplace=True)
df_ai = df_ai.reset_index(drop=False)
df_ai.rename(columns={"file":"y"}, inplace=True)
y = df_ai.pop('y') 
df_ai['y'] = y 
df_ai['rain_FI'] = df_ai.index.map(audios['rain_FI'])

def combinar_nombre_ubicacion(row):
    return f"{row['field_number_PR']}_{row['Filename']}"

# Aplicando la función a cada fila del DataFrame para crear la nueva columna
audios['Filename_'] = audios.apply(combinar_nombre_ubicacion, axis=1)

audios.set_index("Filename_", inplace=True)

In [34]:
def extract_parts(row):
    parts = row.split('_')
    location = parts[0]
    date = parts[1]
    time = parts[2].split('.')[0]  # Eliminar la extensión .WAV
    day = date[-2:]  # Últimos dos caracteres para el día
    hour = time[:2]
    return pd.Series([location, day, hour])

# Aplicar la función a la columna 'y' y crear nuevas columnas
df_ai[['location', 'day', 'hour']] =df_ai['y'].apply(extract_parts)

def define_hour_stage(hour):
    hour = int(hour)
    if 5 <= hour <= 8:
        return 'morning'
    elif 9 <= hour <= 16:
        return 'day'
    else:
        return 'night'

df_ai['hour_stage'] =df_ai['hour'].apply(define_hour_stage)
df_ai.set_index("y", inplace=True, drop=False)
df_ai['cover'] = df_ai.index.map(audios['cover'])
df_ai = df_ai[df_ai['rain_FI'] == 'NO']
df_ai = df_ai.dropna()

/tmp/ipykernel_10771/2020597271.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ai[['location', 'day', 'hour']] =df_ai['y'].apply(extract_parts)
/tmp/ipykernel_10771/2020597271.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ai['hour_stage'] =df_ai['hour'].apply(define_hour_stage)
/tmp/ipykernel_10771/2020597271.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [37]:
df_day = df_ai[df_ai['day'].isin(["05"])]
df_day.columns[0:60]

Index(['ZCR', 'MEANt', 'VARt', 'SKEWt', 'KURTt', 'LEQt', 'BGNt', 'SNRt', 'MED',
       'Ht', 'ACTtFraction', 'ACTtCount', 'ACTtMean', 'EVNtFraction',
       'EVNtMean', 'EVNtCount', 'MEANf', 'VARf', 'SKEWf', 'KURTf', 'NBPEAKS',
       'LEQf', 'ENRf', 'BGNf', 'SNRf', 'Hf', 'EAS', 'ECU', 'ECV', 'EPS',
       'EPS_KURT', 'EPS_SKEW', 'ACI', 'NDSI', 'rBA', 'AnthroEnergy',
       'BioEnergy', 'BI', 'ROU', 'ADI', 'AEI', 'LFC', 'MFC', 'HFC',
       'ACTspFract', 'ACTspCount', 'ACTspMean', 'EVNspFract', 'EVNspMean',
       'EVNspCount', 'TFSD', 'H_Havrda', 'H_Renyi', 'H_pairedShannon',
       'H_gamma', 'H_GiniSimpson', 'RAOQ', 'AGI', 'ROItotal', 'ROIcover'],
      dtype='object')

In [21]:
df_ai

,ZCR,MEANt,VARt,SKEWt,KURTt,LEQt,BGNt,SNRt,MED,Ht,...,RAOQ,AGI,ROItotal,ROIcover,y,location,day,hour,hour_stage,cover
y,,,,,,,,,,,,,,,,,,,,,
RZUA01_20220903_104500.WAV,33413.633333,6.052836e-19,0.000803,0.463214,125.785312,62.065186,-25.232251,3.907083,0.054750,0.854772,...,0.317710,1.371307,699,11.697134,RZUA01_20220903_104500.WAV,RZUA01,03,10,day,pasture
RZUA01_20220903_110000.WAV,44261.500000,-3.324598e-19,0.000308,-0.165748,329.995150,57.900928,-36.107117,10.640441,0.015655,0.654845,...,0.312107,1.388098,778,7.534866,RZUA01_20220903_110000.WAV,RZUA01,03,11,day,pasture
RZUA01_20220903_111500.WAV,39965.483333,3.536716e-19,0.000055,0.165428,28.211735,50.453561,-34.225774,1.959138,0.019441,0.933279,...,0.425797,1.243690,471,5.671398,RZUA01_20220903_111500.WAV,RZUA01,03,11,day,pasture
RZUA01_20220903_113000.WAV,27640.366667,-7.398017e-19,0.002167,0.046234,22.329493,66.379409,-17.784118,2.163875,0.129061,0.940839,...,0.297566,1.288866,522,1.293209,RZUA01_20220903_113000.WAV,RZUA01,03,11,day,pasture
RZUA01_20220903_114500.WAV,30457.066667,1.426097e-18,0.008934,0.011160,11.288357,72.530919,-11.185479,1.877146,0.275884,0.961699,...,0.286141,1.358705,1213,0.604688,RZUA01_20220903_114500.WAV,RZUA01,03,11,day,pasture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RZUH13_20220912_070000.WAV,37127.500000,2.395268e-19,0.000028,-0.123534,4.308900,47.563859,-36.848782,0.656902,0.014373,0.986157,...,0.384166,1.213904,1877,0.920458,RZUH13_20220912_070000.WAV,RZUH13,12,07,morning,pasture
RZUH13_20220912_071500.WAV,42607.800000,-5.329456e-21,0.000021,-0.147668,3.495183,46.161754,-37.072927,0.413386,0.014007,0.990583,...,0.473620,1.213884,1829,1.042269,RZUH13_20220912_071500.WAV,RZUH13,12,07,morning,pasture
RZUH13_20220912_073000.WAV,43009.266667,7.441964e-20,0.000019,-0.064865,4.044839,45.871403,-37.762048,0.553588,0.012939,0.988978,...,0.478226,1.217297,2570,1.936579,RZUH13_20220912_073000.WAV,RZUH13,12,07,morning,pasture


In [38]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

accuracies = []
f1_scores = []
recalls = []
df_day={}
for i in ["03","04","05","06","07","08","09","10","11","12","13"]:
    df_day = df_ai[df_ai['day'].isin([i])]
    X = np.asarray(df_day.iloc[:,0:60])
    y = np.asarray(df_day.loc[:,"cover"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
    clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
    clf_rf.fit(X_train, y_train)
    y_pred_rf = clf_rf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred_rf)
    f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
    recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
    print("Accuracy:", accuracy)
    print("f1:", f1_score)
    print("recall", recall)

    accuracies.append(accuracy)
    f1_scores.append(f1_score)
    recalls.append(recall)

Accuracy: 0.8043478260869565
f1: 0.701019281515238
recall 0.6510204081632653
Accuracy: 0.8
f1: 0.707936507936508
recall 0.66196575604002
Accuracy: 0.7913209955328654
f1: 0.7130842119883525
recall 0.6798427607365841
Accuracy: 0.8048359240069085
f1: 0.7378264538847169
recall 0.7058344628077616
Accuracy: 0.7770114942528735
f1: 0.7041159284809498
recall 0.6767761327405818
Accuracy: 0.7691807542262679
f1: 0.7004273037376937
recall 0.6718954655050208
Accuracy: 0.8245901639344262
f1: 0.7691403821036222
recall 0.7496765406197593
Accuracy: 0.8169542385596399
f1: 0.7708929313645131
recall 0.7478295528703237
Accuracy: 0.8122717311906501
f1: 0.7613150896864211
recall 0.7289883307328733
Accuracy: 0.816
f1: 0.748672141788728
recall 0.7222041651619117
Accuracy: 0.8636363636363636
f1: 0.7668676428366351
recall 0.730451071374949


### Acoustic Indices Data

In [ ]:
X_ai = []
remove = []
s=0
for i in range(len(y_3)):
    
    try:
        X_ai.append(ai.loc[y_3[i]])
    except:
        s+=1
        remove.append(y_3[i])
for i in range(len(remove)):
    y_3.remove(remove[i])
X_ai = np.asarray(X_ai)

labels_ai = []
# audios.set_index("Filename", inplace=True)
for i in range(len(y_3)):
    labels_ai.append(audios.loc[y_3[i], "cover"])
    
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_ai, labels_ai, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy_ai = metrics.accuracy_score(y_test, y_pred_rf)
f1_score_ai = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall_ai = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy_ai)
print("f1:", f1_score_ai)
print("recall", recall_ai)

### Autoencoders Features and Labels using independent segments

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# audios.set_index("Filename", inplace=True)
labels_ae = []
for i in range(len(y_4)):
    labels_ae.append(audios.loc[y_4[i], "cover"])

X_train, X_test, y_train, y_test = train_test_split(X_3, labels_ae, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)

y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)


### Autoencoders Features and Labels using 5 segments of the audio

In [ ]:
X_c = X_3
y_c = y_4

X_batch = np.reshape(X_c, (X_c.shape[0]//5,5,X_c.shape[1]))
y_path = np.reshape(y_c, (y_c.shape[0]//5, 5))

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_batch, y_path, test_size=0.2,random_state=0)

X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[2])
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = y_train.reshape(y_train.shape[0]*y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1])

labels_train = []
for i in range(len(y_train)):
    labels_train.append(audios.loc[y_train[i], "cover"])
labels_test = []
for i in range(len(y_test)):
    labels_test.append(audios.loc[y_test[i], "cover"])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, labels_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(labels_test, y_pred_rf)
f1_score = metrics.f1_score(labels_test, y_pred_rf, average="macro")
recall = metrics.recall_score(labels_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

# Autoencoders Features and Labels using voting labels

In [ ]:
y_pred_rf2 = np.asarray(y_pred_rf)
y_pred_rf2 = np.reshape(y_pred_rf2,(y_pred_rf2.shape[0]//5,5))
y_test2 = np.asarray(labels_test)
y_test2 = np.reshape(y_test2,(y_test2.shape[0]//5,5))

def most_frequent(List):
    return max(set(List), key = List.count)

labels_test2 = []
labels_pred = []
for i in range(len(y_pred_rf2)):
    labels_pred.append(most_frequent(list(y_pred_rf2[i])))
    labels_test2.append(most_frequent(list(y_test2[i])))
accuracy_ae = metrics.accuracy_score(labels_test2, labels_pred)
f1_score_ae = metrics.f1_score(labels_test2, labels_pred, average="macro")
recall_ae = metrics.recall_score(labels_test2, labels_pred, average="macro")

print("Accuracy:", accuracy_ae)
print("f1:", f1_score_ae)
print("recall", recall_ae)

# Autoencoders Features and Labels using mean of segments

In [ ]:
X_New = np.load(f"temporal_zamuro/Features/X_ae_norm_new.npy")
y_New = np.load(f"temporal_zamuro/Features/y_ae_norm_new.npy")
labels_aenew = audios.loc[y_New]
labels_aenew = list(labels_aenew["cover"])


In [ ]:
X_B =np.asarray(X_2)
X_B.shape

In [ ]:
X_New = np.mean(X_B, axis=1)
# X_New = X_New[0:len(y_3)]
# y_New = y_New[0:len(y_3)]

X_New.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_New, labels_aenew, test_size=0.2,random_state=0)
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

In [ ]:
f1_scores = [f1_score_ai, f1_score_ae, f1_score_vggish] # [ai, AE_norm, AE_PCA, AE_UMAP, VGGISH, VGGISH_PCA, VGGISH_UMAP]
accuracy_scores = [accuracy_ai, accuracy_ae, accuracy_vggish]
recall_scores = [recall_ai, recall_ae, recall_vggish]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data for F1 scores, recall, and accuracy
methods = ["Acoustic\nIndices", "AE Normalized", "vggish"]

# Create an array for the x-axis positions
x = np.arange(len(methods))

# Set the figure size
plt.figure(figsize=(11, 6))

# Width of the bars
bar_width = 0.2

# Define professional color palettes
f1_color = 'steelblue'  # You can adjust the color as needed
recall_color = 'indianred'  # You can adjust the color as needed
accuracy_color = 'orange'  # You can adjust the color as needed

# Create the bar chart for F1 scores
bars_f1 = plt.bar(x - bar_width, f1_scores, bar_width, label='F1 Score', color=f1_color)

# Create the bar chart for recall
bars_recall = plt.bar(x, recall_scores, bar_width, label='Recall', color=recall_color)

# Create the bar chart for accuracy
bars_accuracy = plt.bar(x + bar_width, accuracy_scores, bar_width, label='Accuracy', color=accuracy_color)

# Set the y-axis limits
plt.ylim(0.5, 1)

# Set the x-axis labels and their positions
plt.xticks(x, methods, fontsize=14)  # Increase label size

# Set the y-axis label
plt.ylabel('Score')

# Set the title
# plt.title('F1 Score, Recall, and Accuracy Comparison')

# Set the legend with increased size
plt.legend(fontsize=14)  # Increase legend size

# Add values on top of the bars
for bar, value in zip(bars_f1, f1_scores):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height()+0.01, f'{value:.3f}', ha='center', va='bottom', fontsize=11, rotation=90)

for bar, value in zip(bars_recall, recall_scores):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height()+0.01, f'{value:.3f}', ha='center', va='bottom', fontsize=11, rotation=90)

for bar, value in zip(bars_accuracy, accuracy_scores):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height()+0.01, f'{value:.3f}', ha='center', va='bottom', fontsize=11, rotation=90)

# Add a legend
# plt.legend()

# Display the figure

# plt.savefig(f"{root}/Zamuro/temporal_zamuro/zamuro_classification_results/zamuro_classification.pdf", format="pdf")